In [14]:
import wikipedia
import re
import os
import time
import json
from tqdm import tqdm

In [15]:
# Define the path for the tracking file
TRACKING_FILE = "data/scraped_articles.json"

# Load previously scraped articles from JSON file
def load_scraped_articles():
    if os.path.exists(TRACKING_FILE):
        try:
            with open(TRACKING_FILE, 'r', encoding='utf-8') as f:
                data = json.load(f)
                return set(data.get('articles', []))
        except Exception as e:
            print(f"Error loading tracking file: {e}")
            return set()
    else:
        # Create empty tracking file if it doesn't exist
        with open(TRACKING_FILE, 'w', encoding='utf-8') as f:
            json.dump({"articles": []}, f)
        return set()

# Save scraped articles to JSON file
def save_scraped_articles(scraped_articles):
    try:
        with open(TRACKING_FILE, 'w', encoding='utf-8') as f:
            json.dump({"articles": list(scraped_articles)}, f, indent=2)
        return True
    except Exception as e:
        print(f"Error saving tracking file: {e}")
        return False

In [16]:
def extract_wikipedia_text(article_titles, output_dir="data/articles", sleep_time=2):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Load previously scraped articles
    scraped_articles = load_scraped_articles()
    
    # Set Wikipedia language
    wikipedia.set_lang("en")
    
    results = {}
    
    for title in tqdm(article_titles):
        try:
            # Get the article content
            page = wikipedia.page(title, auto_suggest=False)
            
            # Extract the content
            content = page.content
            
            # Clean the text
            # Remove section headings (== Section title ==)
            content = re.sub(r'==+\s*[^=]+\s*==+', '', content)
            
            # Remove citations like [1], [2], etc.
            content = re.sub(r'\[\d+\]', '', content)
            
            # Remove text inside parentheses (often contains pronunciation guides or additional info)
            # content = re.sub(r'\([^)]*\)', '', content)  # Optional, may remove important content
            
            # Remove extra whitespace
            content = re.sub(r'\s+', ' ', content).strip()
            
            # Check if content has at least 3 periods (dots)
            if content.count('.') < 3:
                print(f"Rejecting '{title}' - insufficient sentences (less than 3 periods)")
                continue

            # Save to file
            clean_title = re.sub(r'[^\w\s-]', '', title).strip().replace(' ', '_')
            file_path = os.path.join(output_dir, f"{clean_title}.txt")
            
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(content)
            
            # Store in dictionary
            results[title] = content
            
            # Add to scraped articles and save tracking file
            scraped_articles.add(title)
            save_scraped_articles(scraped_articles)
            print(f"Saved article: '{title}' and updated tracking file")
            
            # Sleep to avoid rate limiting
            time.sleep(sleep_time)
            
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"DisambiguationError for '{title}': {e}")
            print(f"Suggestions: {e.options[:5]}")
            
        except wikipedia.exceptions.PageError:
            print(f"PageError: '{title}' does not match any pages. Try another title!")
            
        except Exception as e:
            print(f"Error processing '{title}': {e}")
    
    return results

In [17]:
from utils.files import *
# Read the file
wiki_articles_names = read_file("data/wikiArticleTitleList.txt")

In [18]:
print(wiki_articles_names[:50])

!
!!
!!!
!!!Fuck_You!!!
!!!Fuck_You!!!_And_Then_So


In [19]:
def is_ascii(s):
    try:
        s.encode('ascii').decode('ascii')
    except UnicodeEncodeError:
        return False
    else:
        return True

In [20]:
wiki_articles_names = wiki_articles_names.split("\n")

In [21]:
print(wiki_articles_names[:5])  # Print the first 5 article titles for verification

['!', '!!', '!!!', '!!!Fuck_You!!!', '!!!Fuck_You!!!_And_Then_Some']


In [22]:
# Filter articles that have already been scraped
scraped_articles = load_scraped_articles()
print(f"Found {len(scraped_articles)} previously scraped articles")

# Filter out already scraped articles
wiki_articles_names = [name for name in wiki_articles_names if name not in scraped_articles]
print(f"{len(wiki_articles_names)} articles remaining after removing already scraped ones")

Found 395 previously scraped articles
14846581 articles remaining after removing already scraped ones


In [23]:
def is_valid_windows_filename(filename):
    reserved_chars = r'[<>:"/\\|?*]'
    # Reserved names in Windows
    reserved_names = [
        "CON", "PRN", "AUX", "NUL", "COM1", "COM2", "COM3", "COM4",
        "COM5", "COM6", "COM7", "COM8", "COM9", "LPT1", "LPT2", "LPT3",
        "LPT4", "LPT5", "LPT6", "LPT7", "LPT8", "LPT9"
    ]

    # Check for invalid characters
    if re.search(reserved_chars, filename):
        return False

    # Check for reserved names (case-insensitive)
    if filename.upper() in reserved_names:
        return False

    # Check if the filename ends with a period or space
    if filename.endswith(".") or filename.endswith(" "):
        return False

    # Check if the filename is too long (Windows has a limit of 255 characters for the full path)
    if len(filename) > 255:
        return False

    return True

In [24]:
wiki_articles_names = [name for name in wiki_articles_names if is_valid_windows_filename(name)]

In [25]:
import random
random.shuffle(wiki_articles_names)

N_ARTICLES = 1000

wiki_articles_names = wiki_articles_names[:N_ARTICLES]  # Limit to 1000 articles for testing

In [26]:
extracted_texts = extract_wikipedia_text(wiki_articles_names)
    
# Print summary
print("\nExtraction complete!")
print(f"Successfully extracted {len(extracted_texts)} articles")

# Preview first 200 characters of first article
if extracted_texts:
    first_title = list(extracted_texts.keys())[0]
    print(f"\nPreview of '{first_title}':")
    print(extracted_texts[first_title][:200] + "...")

  0%|          | 0/1000 [00:00<?, ?it/s]

Saved article: 'Jewish_Channel' and updated tracking file


  0%|          | 2/1000 [00:04<31:20,  1.88s/it]  

Error processing 'C%C3%A9sar_Roux': 'fullurl'
Saved article: 'Mesarfelta' and updated tracking file


  0%|          | 3/1000 [00:07<41:43,  2.51s/it]

...

Saved article: 'Johnny_in_the_Valley_of_the_Giants' and updated tracking file


100%|█████████▉| 999/1000 [46:52<00:03,  3.07s/it]

Saved article: 'Lakeview_Drive' and updated tracking file


100%|██████████| 1000/1000 [46:55<00:00,  2.82s/it]


Extraction complete!
Successfully extracted 690 articles

Preview of 'Jewish_Channel':
The Jewish Channel (TJC) is a cable television channel available on Comcast, iO Optimum Cablevision, Time Warner Cable, Verizon FiOS, RCN, Frontier Communications, and Cox Cable. It was launched by Co...
